In [1]:
import motile
import networkx as nx
from LineageTree import lineageTree
import utils
import os
path = r'/localscratch/DL4MIA_2024/project/data/tracks'
fname = r'GN692_125kPa002_20231017-142851.xml'
image_path = r'/group/dl4miacourse/TRNs_calcium'

crop = 200

file = os.path.join(path, fname)
lt = lineageTree(file, file_type='trackmate')

nx_graph = utils.to_motile(lt, crop=crop, max_dist=20)

graph = motile.TrackGraph(nx_graph)

Extracting candidate edges
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]


100%|██████████| 199/199 [00:00<00:00, 36451.50it/s]


In [2]:
import motile

# create a motile solver
solver = motile.Solver(graph)

In [3]:
from motile.costs import EdgeDistance

solver.add_costs(
    EdgeDistance(
        weight=1,
        position_attribute="pos",
        constant=-20))


In [4]:
from motile.constraints import MaxParents, MaxChildren

solver.add_constraints(MaxParents(1))
solver.add_constraints(MaxChildren(1))

In [5]:
from motile.costs import Appear

solver.add_costs(Appear(constant=1.0))

In [6]:
solution = solver.solve()
solution_graph = solver.get_selected_subgraph(solution=solution)

Could not create Gurobi backend: Gurobi error in ilpy/impl/solvers/GurobiBackend.cpp:22: PIP license can only be used from gurobipy interface


TODO:
add results to napari tracks layer and/or points layer and start viewer
Add raw data to image layer

In [7]:
from motile_toolbox.visualization.napari_utils import to_napari_tracks_layer
from motile_toolbox.candidate_graph.graph_to_nx import graph_to_nx

tracks_layer = to_napari_tracks_layer(
    graph_to_nx(solution_graph), frame_key="t", location_key="pos"
)
tracks_layer



(array([[  0.        ,   1.        , 631.34260294, 598.91042525,
           0.        ],
        [  1.        ,   1.        , 695.47751831, 569.80989695,
           0.        ],
        [  1.        ,   2.        , 686.30997148, 569.10058725,
           0.        ],
        ...,
        [ 41.        , 198.        , 511.62313775, 740.48913203,
           0.        ],
        [ 42.        , 199.        , 674.83219342, 509.06601739,
           0.        ],
        [ 41.        , 199.        , 494.28132263, 748.13482187,
           0.        ]]),
 {},
 {})

In [8]:
import napari

viewer = napari.Viewer()


In [9]:
import tifffile
import glob
import zarr
image_path = '/group/dl4miacourse/projects/OptiMates/TRNs/batch1'
image_fname = r'GN692_125kPa002_20231017-142851.tif'
lazyimage = tifffile.imread(os.path.join(image_path, image_fname), aszarr=True)

z = zarr.open(lazyimage, mode='r')
image = z[:crop]

viewer.add_image(image,
                channel_axis=1,
                name=["GCamp", "TagRFP"],
                colormap=["green", "red"], 
                blending='additive')
viewer.show()

In [10]:
reformat = lambda tracks_layer: tracks_layer[0][:, [0, 1, 3, 2]]
tracks = reformat(tracks_layer)
print(len(set(tracks[:, 0])), 'tracks detected')
viewer.add_tracks(tracks, name="tracks2")

viewer.show()

43 tracks detected


In [11]:
import ilpy
from motile.variables import NodeAppear

class LimitNumTracks(motile.constraints.Constraint):

  def __init__(self, num_tracks):
      self.num_tracks = num_tracks

  def instantiate(self, solver):

      appear_indicators = solver.get_variables(NodeAppear)

      constraint = ilpy.Constraint()
      for appear_indicator, index in appear_indicators.items():
        constraint.set_coefficient(index, 1.0)
      constraint.set_relation(ilpy.Relation.LessEqual)
      constraint.set_value(self.num_tracks)

      return [constraint]
maxTracks = 3 
solver.add_constraints(LimitNumTracks(maxTracks))


In [12]:
solution = solver.solve()
solution_graph = solver.get_selected_subgraph(solution=solution)
tracks_layer = to_napari_tracks_layer(
    graph_to_nx(solution_graph), frame_key="t", location_key="pos"
)
tracks = reformat(tracks_layer)
print(len(set(tracks[:, 0])), 'tracks detected')
viewer.add_tracks(tracks, name="contrains, max " + str(maxTracks))


Could not create Gurobi backend: Gurobi error in ilpy/impl/solvers/GurobiBackend.cpp:22: PIP license can only be used from gurobipy interface
3 tracks detected


<Tracks layer 'contrains, max 3' at 0x7f987d4f77c0>

In [13]:
import importlib
importlib.reload(utils)


<module 'utils' from '/localscratch/DL4MIA_2024/project/OptiMates/src/OptiMates/utils.py'>

In [15]:
import numpy as np
from utils import read_points_fromLT

points = read_points_fromLT(lt, crop=crop)
viewer.add_points(points, name='cells')

{'time_nodes': {0: set(), 1: {81337, 81339, 81334}, 2: {81345, 81348, 81351}, 3: {81393, 81398}, 4: {81344}, 5: {81367}, 6: {81372}, 7: {81403}, 8: {81408, 81404}, 9: {81443, 81444, 81446}, 10: {81376, 81373, 81374}, 11: {81441, 81442, 81437}, 12: {81353, 81356, 81357}, 13: {81378, 81381, 81382}, 14: {81450, 81451, 81452}, 15: {81456, 81457, 81458}, 16: {81361, 81362, 81363}, 17: {81434, 81435, 81436}, 18: {81412, 81413, 81414}, 19: {81392, 81390, 81391}, 20: {81424, 81422, 81423}, 21: {81384, 81385, 81386}, 22: {81426, 81428, 81429, 81430}, 23: {81417, 81418, 81419}, 24: {81538, 81539, 81540}, 25: {81561, 81562}, 26: {81526, 81527}, 27: {81480, 81481}, 28: {81504, 81503}, 29: {81521, 81522}, 30: {81552, 81553}, 31: {81470, 81471}, 32: {81460, 81461}, 33: {81467, 81468}, 34: {81556, 81557, 81558}, 35: {81475, 81476, 81477}, 36: {81507, 81508, 81509}, 37: {81530, 81531}, 38: {81512, 81513}, 39: {81489, 81490}, 40: {81536, 81535}, 41: {81499, 81500}, 42: {81464, 81465}, 43: {81484, 81485

<Points layer 'cells' at 0x7f98a29f73a0>

In [16]:
skip_frames = 10
nx_graph = utils.to_motile(lt, crop=crop, max_dist=100, max_skip_frames=skip_frames)
graph = motile.TrackGraph(nx_graph)
solver = motile.Solver(graph)

solver.add_costs(
    EdgeDistance(
        weight=1,
        position_attribute="pos",
        constant=-100))

solver.add_constraints(MaxParents(1))
solver.add_constraints(MaxChildren(1))
#solver.add_costs(Appear(constant=1.0))


maxTracks = 3 
solver.add_constraints(LimitNumTracks(maxTracks))

solution = solver.solve()

solution_graph = solver.get_selected_subgraph(solution=solution)
tracks_layer = to_napari_tracks_layer(
    graph_to_nx(solution_graph), frame_key="t", location_key="pos"
)

tracks = reformat(tracks_layer)
print(len(set(tracks[:, 0])), 'tracks detected')
viewer.add_tracks(tracks, name="contrains, max " + str(maxTracks) + 'skip frames' + str(skip_frames))




Extracting candidate edges
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]


100%|██████████| 199/199 [00:00<00:00, 4744.77it/s]

Could not create Gurobi backend: Gurobi error in ilpy/impl/solvers/GurobiBackend.cpp:22: PIP license can only be used from gurobipy interface


3 tracks detected


<Tracks layer 'contrains, max 3skip frames10' at 0x7f98a26d5570>

Traceback (most recent call last):
  File "/localscratch/miniforge3/envs/motile/lib/python3.10/site-packages/napari_animation/_qt/frame_widget.py", line 66, in _update_animation_ease
    active_keyframe.ease = self.get_easing_func()
AttributeError: 'NoneType' object has no attribute 'ease'
Traceback (most recent call last):
  File "/localscratch/miniforge3/envs/motile/lib/python3.10/site-packages/napari_animation/_qt/frame_widget.py", line 66, in _update_animation_ease
    active_keyframe.ease = self.get_easing_func()
AttributeError: 'NoneType' object has no attribute 'ease'

(python:53070): GVFS-RemoteVolumeMonitor-WARNING **: 16:18:54.570: remote volume monitor with dbus name org.gtk.vfs.GoaVolumeMonitor is not supported


Rendering frames...


100%|██████████| 76/76 [00:02<00:00, 27.13it/s]
